In [1]:
# You have to click the restart kernel button after each time you run it otherwise the graphics get really weird
# If you want to train the AI faster without graphics, go into the PoolGame.py file and change the GymMiniGame default render_screen value to False

import tf_agents.agents
from tf_agents.environments import tf_py_environment
from tf_agents.policies import greedy_policy, random_tf_policy
from tf_agents.agents import DdpgAgent
from tf_agents.environments import suite_gym, utils
from gym.envs.registration import register
from tf_agents.replay_buffers  import tf_uniform_replay_buffer
import tensorflow as tf
from tf_agents.utils import common
from tf_agents.trajectories import trajectory
import pickle

2023-03-03 08:53:33.450500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
register(
    id='PoolGame-v0',
    entry_point='PoolGame:GymMiniGame',
    max_episode_steps=300,
)

In [3]:
gym_env = suite_gym.load("PoolGame-v0")


/Users/23crowley/Documents/Python/PoolGame/venv/lib/python3.10/site-packages/gym/envs/registration.py:595: UserWarning: WARN: Overriding environment PoolGame-v0
  logger.warn(f"Overriding environment {id}")


In [4]:
tf_env = tf_py_environment.TFPyEnvironment(gym_env)

global_step = tf.compat.v1.train.get_or_create_global_step()



2023-03-03 08:55:23.789462: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Hyper parameters
gamma = 0.995
actor_learning_rate = 1e-4
critic_learning_rate = 1e-4
actor_fully_connected_layer_units = (400, 300)
critic_obs_fc_layers=(400,)
critic_action_fc_layers=None
critic_joint_fc_layers=(300,)
# Params for collect
initial_collect_steps=990
num_iterations=100
collect_steps_per_iteration=10
replay_buffer_capacity=50_000
batch_size = 64

# Params for summaries and logging
log_interval = 100
save_interval = 500


ou_stddev=0.2
ou_damping=0.15
target_update_tau=0.05
target_update_period=5
dqda_clipping=None
td_errors_loss_fn=tf.compat.v1.losses.huber_loss
reward_scale_factor=1.0
gradient_clipping=None

load_saved_model = True     # False if you want to run it from scratch

In [6]:
# Actor Network
actor_network = tf_agents.agents.ddpg.actor_network.ActorNetwork(tf_env.observation_spec(), tf_env.action_spec(), fc_layer_params=actor_fully_connected_layer_units)


In [7]:
# Critic Network
critic_network = tf_agents.agents.ddpg.critic_network.CriticNetwork((tf_env.observation_spec(), tf_env.action_spec()), joint_fc_layer_params=critic_joint_fc_layers)

In [8]:
tf_agent = DdpgAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    actor_network=actor_network,
    critic_network=critic_network,
    train_step_counter=global_step,
    actor_optimizer=tf.compat.v1.train.AdamOptimizer(
        learning_rate=actor_learning_rate),
    critic_optimizer=tf.compat.v1.train.AdamOptimizer(
        learning_rate=critic_learning_rate),
    gamma=gamma,
    ou_stddev=ou_stddev,
    ou_damping=ou_damping,
    target_update_tau=target_update_tau,
    target_update_period=target_update_period,
    dqda_clipping=dqda_clipping,
    td_errors_loss_fn=td_errors_loss_fn,
    reward_scale_factor=reward_scale_factor,
    gradient_clipping=gradient_clipping,

)

tf_agent.initialize()

eval_policy = greedy_policy.GreedyPolicy(tf_agent.policy)
random_policy = random_tf_policy.RandomTFPolicy(tf_env.time_step_spec(), tf_env.action_spec())
collect_policy = tf_agent.collect_policy

In [9]:
# Create Replay Buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(tf_agent.collect_data_spec,
                                                               batch_size=tf_env.batch_size,
                                                               max_length=replay_buffer_capacity)

# Create Checkpointer
train_checkpointer = common.Checkpointer(
    ckpt_dir="SaveFiles/train",
    agent=tf_agent,
    policy=tf_agent.policy,
    replay_buffer=replay_buffer,
    global_step=global_step)

train_checkpointer.initialize_or_restore()


In [10]:
def collect_step(environment, policy, buffer, reward_list):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    reward_list.append(next_time_step[1].numpy()[0])
    traj = trajectory.from_transition(time_step, action_step, next_time_step)

    # Add trajectory to the replay buffer
    buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps, reward_list):
    for _ in range(steps):
      collect_step(env, policy, buffer, reward_list)

In [11]:
def save_results(reward_list):
    with open("SaveFiles/rewards.pickle", 'wb') as f:
        pickle.dump(reward_list, f)

def load_results():
    with open("SaveFiles/rewards.pickle", 'rb') as f:
        reward_list = pickle.load(f)
    return reward_list

In [12]:
# Collect initial replay data.

if not load_saved_model:
    rewards = []
    collect_data(tf_env, random_policy, replay_buffer, steps=initial_collect_steps, reward_list=rewards)
else:
    print("Loading Saved Reward Data")
    rewards = load_results()

# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

Loading Saved Reward Data
Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [13]:
iterator = iter(dataset)

tf_agent.train = common.function(tf_agent.train)


In [ ]:
for _ in range(num_iterations):
    collect_data(tf_env, tf_agent.collect_policy, replay_buffer, collect_steps_per_iteration, rewards)

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    train_loss = tf_agent.train(experience).loss

    step = len(rewards)

    if step % log_interval == 0:
        print(f'****** step {step} ******: '
              f'\nlast 100 reward = {sum(rewards[max(len(rewards) - 100, 0):])/min(len(rewards), 100)}'
              f'\nlast 1000 reward = {sum(rewards[max(len(rewards) - 1000, 0):])/min(len(rewards), 1000)}'
              f'\nlast 5000 reward = {sum(rewards[max(len(rewards) - 5000, 0):])/min(len(rewards), 5000)}'
              f'\ntotal average reward = {sum(rewards)/len(rewards)}\n')

    if step % save_interval == 0:
        train_checkpointer.save(global_step)
        save_results(rewards)
        print("\nModel Saved\n")



In [ ]:
import matplotlib.pyplot as plt
reward_lumps = [sum(rewards[i*1000:(i+1)*1000])/1000 for i in range(len(rewards)//1000)]
plt.plot(range(1000, len(rewards), 1000), reward_lumps)
yl = plt.xlabel("Number of Shots Taken")
xl = plt.ylabel("Success Rate")